In [1]:
# ===============================
# Step 1: Install required packages
# ===============================
# openai-agents : OpenAI Agents Framework
# openai        : OpenAI API client
# requests      : HTTP requests to fetch product data from API endpoints
# nest_asyncio  : Enable nested async calls in Jupyter / Colab
%pip install --quiet openai-agents openai requests nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 5.5 MB/s eta 0:00:00


In [2]:
# ===============================
# Step 2: Import required libraries
# ===============================
import os
import requests
from getpass import getpass
from openai import OpenAI

In [3]:
# ===============================
# Step 3: Set OpenAI API key
# ===============================
# Replace with your own OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass("🔑 Paste your OpenAI API key (starts with sk-...): ")
print("✅ New API key set successfully!")
print("Key starts with:", os.getenv("OPENAI_API_KEY")[:5])  # Only first 5 characters shown for security


🔑 Paste your OpenAI API key (starts with sk-...): ··········
✅ New API key set successfully!
Key starts with: sk-pr


In [4]:
# ===============================
# Step 4: Initialize OpenAI client
# ===============================
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [5]:
# ===============================
# Step 5: Fetch products from API
# ===============================
def fetch_products():
    """
    Function to fetch products from API endpoint.
    Returns a list of products (JSON) if successful, else empty list.
    """
    url = "https://hackathon-apis.vercel.app/api/products"  # API endpoint
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return []

# Test fetch
products = fetch_products()
print("✅ Total products fetched:", len(products))
print("Example product:", products[0])

✅ Total products fetched: 24
Example product: {'name': 'The Poplar suede sofa', 'description': 'A timeless design, with premium materials features as one of our most popular and iconic pieces. The dandy chair is perfect for any stylish living space with beech legs and lambskin leather upholstery.', 'image': 'https://cdn.sanity.io/images/ri847jqu/production/9b6a4fc8c65bbb4e5793fb0e1116b510d73dc9e8-630x375.png', '_id': '65453ffd-e476-4b6b-a388-7e3de1bb632a', 'features': ['Premium material', 'Handmade upholster', 'Quality timeless classic'], 'dimensions': {'width': '110cm', 'height': '110cm', 'depth': '50cm'}, 'category': {'name': 'Tableware', 'slug': 'tableware'}, 'price': 980, 'tags': ['popular products']}


In [6]:
# ===============================
# Step 6: Function to filter products
# ===============================
def filter_products(products, max_price=None, keyword=None):
    """
    Filter products by maximum price and/or keyword in product name.
    """
    results = products

    if max_price:
        results = [p for p in results if p.get("price", 0) <= max_price]
    if keyword:
        results = [p for p in results if keyword.lower() in p.get("name", "").lower()]

    return results


In [7]:
# ===============================
# Step 7: Define shopping agent query function
# ===============================
def shopping_agent_query(query, products):
    """
    Function to interact with OpenAI model as a shopping assistant.
    Returns AI-generated response based on product list.
    """
    context = "You are a shopping assistant. Answer based on the product list."

    # First 10 products used as context
    product_titles = "\n".join([f"{p['name']} - ${p['price']}" for p in products[:10]])

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": context},
            {"role": "user", "content": f"{query}\nHere are some products:\n{product_titles}"}
        ]
    )

    return response.choices[0].message.content

In [8]:
# ===============================
# Step 8: Test shopping agent
# ===============================
query1 = "Show me products under $300"
query2 = "Find me a chair"

print("Query 1 Result:\n", shopping_agent_query(query1, products))
print("\nQuery 2 Result:\n", shopping_agent_query(query2, products))

Query 1 Result:
 Here are the products under $300:

1. The Steel Chair - $250
2. Rustic Vase Set - $210
3. Bed - $250
4. Wood Chair - $100
5. The Lucky Lamp - $200
6. Pure Aura - $280 (Note: This is exactly $280 and may be considered depending on your specific needs)
7. Zen Table - $250

Let me know if you need more information about any of these products!

Query 2 Result:
 Here are the chairs from your product list:

1. The Steel Chair - $250
2. Wood Chair - $100

You can choose between the Steel Chair or the Wood Chair based on your preference!
